# Overview

We have a set of data, and a set of known values that a subset of the data should match.
The goal is to add two columns to the data.
The first column should contain for each row a list of column names whose data in that row differ from the known values.
The second column should contain for each row a boolean that is True if the values differ from the known values, and False otherwise.

Import Pandas

In [ ]:
import pandas as pd

Set up our data

In [ ]:
data = pd.DataFrame.from_dict({
    "name_h": ["ha", "ha", "hb"],
    "name_l": ["la", "la", "lb"],
    "hc1": [1,1,2],
    "hc2": [2,3,4],
    "lc1": [5,5,6],
    "lc2": [6,5,6],
})
data

And set up our known values

In [ ]:
knowns_dict = {
    "ha": {
        "hc1": 1,
        "hc2": 2,
    },
    "hb": {
        "hc1": 3,
        "hc2": 4,
    },
    "la": {
        "lc1": 5,
        "lc2": 6,
    },
    "lb": {
        "lc1": 7,
        "lc2": 8,
    },
}
knowns = pd.DataFrame.from_dict(knowns_dict).transpose()
knowns

Merge our data with our knowns to get a known df that is the same shape/size as our data, that way we can compare them directly

In [ ]:
# Merge based on the heavy names
knowns_h = data.merge(knowns, how="left", right_index=True, left_on="name_h", suffixes=["_data_h", None])
knowns_h

In [ ]:
# Separately merge based on the light names
knowns_l = data.merge(knowns, how="left", right_index=True, left_on="name_l", suffixes=["_data_l", None])
knowns_l

In [ ]:
# Combine the heavy and light merges to get the columns from both
# Use DataFrame.combine_first so that values aren't overwritten by NaNs
# Reindex by the data index and knowns columns so that we only keep the columns/rows we care about
known_to_compare = knowns_h.combine_first(knowns_l).reindex(index=data.index, columns=knowns.columns)
known_to_compare

In [ ]:
# Start with only the relevant columns from our data
df = data[known_to_compare.columns]
df

In [ ]:
# Compare to the known values - this produces a multi-level-index on the columns,
# and two columns for each original column: "self" and "other"
df = df.compare(known_to_compare, align_axis=1)
df

Take the subset of the data that _corresponds_ (not _matches_) with the known values.
Compare the subset of the data with the known values.
Compute a list of the columns that did not match for each row.
Add the result to the original data.

In [ ]:
# Start with only the relevant columns from our data
df = data[known_to_compare.columns]
df

In [ ]:
# Compare to the known values - this produces a multi-level-index on the columns,
# and two columns for each original column: "self" and "other"
df = df.compare(known_to_compare, align_axis=1)
df

In [ ]:
# Get rid of the "self" column so that we only have one column per original column, and we don't need the actual values
df = df.drop("self", level=1, axis=1)
df

In [ ]:
# Get rid of the multi-level-index on the columns, specifically the level that "self" and "other" were in
df = df.droplevel(1, axis=1)
df

In [ ]:
# Use the column names as an index
df = df.stack(future_stack=True)
df

In [ ]:
# Reset the index to put the index values into columns so that we can work with them
# The reset_index method produces two columns: level_0 and level_1
# level_0 is our original row index from data and level_1 is our column name that compared as different
df = df.reset_index()
df

In [ ]:
# Drop NA values to get rid of rows that all matched the knowns
df = df.dropna(how="any")
df

In [ ]:
differing_columns = (
    df
    # For each row (the "level_0" column) take the column name (in the "level_1" column)
    .groupby("level_0")["level_1"]
    # Aggregate the column names into a list for each row
    .agg(list)
    # Rename the resulting series to make sense instead of "level_1"
    .rename("Differing Columns")
)
differing_columns

In [ ]:
# Concatenate the "differing" series with the original data
dfo = pd.concat([data, differing_columns], axis=1)
dfo

In [ ]:
# Create the "Differs" column as True if "Differing Columns" has columns, otherwise False
dfo["Differs"] = dfo["Differing Columns"].notna()
# Clean up the new column to make it pretty by replacing NaNs with an empty string
dfo.fillna({"Differing Columns": ""}, inplace=True)
dfo